# Using CDP Databases
Methods for retrieving open access data.

A database schema diagram for production instances of CDP may be found [here](https://github.com/CouncilDataProject/cdptools/blob/master/docs/resources/database_diagram.pdf).

### Connecting to the database

CDP Seattle uses Firebase's 'Cloud Firestore' to store our data. However, a properly setup database host and associated database module _should_ have the same functionality.

**Note:** This notebook connects to the staging instance of Seattle's Firestore database. To use production data, connect to `cdp-seattle`.

In [1]:
from cdptools.databases.cloud_firestore_database import CloudFirestoreDatabase

db = CloudFirestoreDatabase("stg-cdp-seattle")
db

<CloudFirestoreDatabase [stg-cdp-seattle]>

### Retrieving a single item
If you know the id of an item in a table, please use the `select_row_by_id` function provided.

In [2]:
event = db.select_row_by_id(table="event", id="1408bf08-d6c0-4ab7-96de-e1ef2294f075")
event

{'event_id': '1408bf08-d6c0-4ab7-96de-e1ef2294f075',
 'created': datetime.datetime(2019, 7, 21, 0, 29, 25, 1338),
 'legistar_event_link': 'https://seattle.legistar.com/MeetingDetail.aspx?LEGID=4033&GID=393&G=FFE3B678-CEF6-4197-84AC-5204EA4CFC0C',
 'body_id': 'ad388d06-06d4-4af5-a2f8-901e977cfaa7',
 'source_uri': 'http://www.seattlechannel.org/mayor-and-council/city-council/2018/2019-planning-land-use-and-zoning-committee?videoid=x105878',
 'legistar_event_id': 4033,
 'event_datetime': datetime.datetime(2019, 7, 17, 9, 30),
 'agenda_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/7/4033_A_Planning%2C_Land_Use%2C_and_Zoning_Committee_19-07-17_Committee_Agenda.pdf',
 'minutes_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/7/4033_M_Planning%2C_Land_Use%2C_and_Zoning_Committee_19-07-17_Committee_Minutes.pdf',
 'video_uri': 'http://video.seattle.gov:8080/media/council/plan_071719_2511923V.mp4'}

### Retrieving many items from a table

You may not know the id's of items you are looking for. In that case, use the `select_rows_as_list` function provided.

In [3]:
events = db.select_rows_as_list(table="event")
events[0]

{'event_id': '1408bf08-d6c0-4ab7-96de-e1ef2294f075',
 'legistar_event_id': 4033,
 'event_datetime': datetime.datetime(2019, 7, 17, 9, 30),
 'agenda_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/7/4033_A_Planning%2C_Land_Use%2C_and_Zoning_Committee_19-07-17_Committee_Agenda.pdf',
 'minutes_file_uri': 'http://legistar2.granicus.com/seattle/meetings/2019/7/4033_M_Planning%2C_Land_Use%2C_and_Zoning_Committee_19-07-17_Committee_Minutes.pdf',
 'video_uri': 'http://video.seattle.gov:8080/media/council/plan_071719_2511923V.mp4',
 'created': datetime.datetime(2019, 7, 21, 0, 29, 25, 1338),
 'legistar_event_link': 'https://seattle.legistar.com/MeetingDetail.aspx?LEGID=4033&GID=393&G=FFE3B678-CEF6-4197-84AC-5204EA4CFC0C',
 'body_id': 'ad388d06-06d4-4af5-a2f8-901e977cfaa7',
 'source_uri': 'http://www.seattlechannel.org/mayor-and-council/city-council/2018/2019-planning-land-use-and-zoning-committee?videoid=x105878'}

### Joining with other tables

In the above event results, notice that a `body_id` is returned for each event. To attach body details to this we can use the python package `pandas` and query the `body` table. Let's first put each of the query results into `pandas.DataFrame` objects.

In [4]:
import pandas as pd

In [5]:
events = pd.DataFrame(events)
events.head()

,agenda_file_uri,body_id,created,event_datetime,event_id,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri
0,http://legistar2.granicus.com/seattle/meetings...,ad388d06-06d4-4af5-a2f8-901e977cfaa7,2019-07-21 00:29:25.001338,2019-07-17 09:30:00,1408bf08-d6c0-4ab7-96de-e1ef2294f075,4033,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/pl...
1,http://legistar2.granicus.com/seattle/meetings...,30a859c2-3755-4754-a108-c84476bfe886,2019-07-20 23:44:41.348645,2019-06-24 10:30:00,1d0c6214-1343-463d-b0ab-03a3d5e36c5d,4008,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ho...
2,http://legistar2.granicus.com/seattle/meetings...,b6792cc8-83cb-4c56-b38f-4997f16eb2ba,2019-07-21 00:16:04.078408,2019-06-25 14:00:00,276c6722-b35c-4341-b7fe-304d874a17ef,4012,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/hu...
3,http://legistar2.granicus.com/seattle/meetings...,5f5d3cd0-49e3-4700-9341-95523a10fd49,2019-07-21 00:21:31.105103,2019-06-27 12:00:00,294015e9-ee34-4de3-80d3-27175a127275,4013,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ho...
4,http://legistar2.granicus.com/seattle/meetings...,52aaf5d7-ed25-44c6-a236-d6ae915ff432,2019-07-20 23:28:27.305315,2019-07-15 10:30:00,2cbb7f4b-2e19-4f44-a99e-a0dcebdb3ac3,4029,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ar...


In [6]:
bodies = db.select_rows_as_list("body")
bodies = pd.DataFrame(bodies)
bodies.head()

,body_id,created,description,name
0,0239d692-4514-4d62-bdcb-3b0c470dee5d,2019-07-20 23:49:05.063092,None,Finance and Neighborhoods Committee
1,2448987c-36a0-48f3-a7ad-c76d20369d0c,2019-07-20 23:27:44.607321,None,"Civil Rights, Utilities, Economic Development,..."
2,30a859c2-3755-4754-a108-c84476bfe886,2019-07-20 23:44:41.123429,None,Select Committee on Homelessness and Housing A...
3,4364bde3-f1a4-47b7-9cf2-478296781681,2019-07-20 23:34:08.264936,None,"Civic Development, Public Assets, and Native C..."
4,52aaf5d7-ed25-44c6-a236-d6ae915ff432,2019-07-20 23:28:27.107855,None,Select Committee on Civic Arenas


In [7]:
expanded_event_details = events.merge(bodies, left_on="body_id", right_on="body_id", suffixes=("_event", "_body"))
expanded_event_details.head()

,agenda_file_uri,body_id,created_event,event_datetime,event_id,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri,created_body,description,name
0,http://legistar2.granicus.com/seattle/meetings...,ad388d06-06d4-4af5-a2f8-901e977cfaa7,2019-07-21 00:29:25.001338,2019-07-17 09:30:00,1408bf08-d6c0-4ab7-96de-e1ef2294f075,4033,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/pl...,2019-07-21 00:29:24.783820,None,"Planning, Land Use, and Zoning Committee"
1,http://legistar2.granicus.com/seattle/meetings...,30a859c2-3755-4754-a108-c84476bfe886,2019-07-20 23:44:41.348645,2019-06-24 10:30:00,1d0c6214-1343-463d-b0ab-03a3d5e36c5d,4008,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ho...,2019-07-20 23:44:41.123429,None,Select Committee on Homelessness and Housing A...
2,http://legistar2.granicus.com/seattle/meetings...,b6792cc8-83cb-4c56-b38f-4997f16eb2ba,2019-07-21 00:16:04.078408,2019-06-25 14:00:00,276c6722-b35c-4341-b7fe-304d874a17ef,4012,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/hu...,2019-07-21 00:16:03.860927,None,"Human Services, Equitable Development, and Ren..."
3,http://legistar2.granicus.com/seattle/meetings...,5f5d3cd0-49e3-4700-9341-95523a10fd49,2019-07-21 00:21:31.105103,2019-06-27 12:00:00,294015e9-ee34-4de3-80d3-27175a127275,4013,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ho...,2019-07-21 00:13:11.547208,None,"Housing, Health, Energy, and Workers’ Rights C..."
4,http://legistar2.granicus.com/seattle/meetings...,5f5d3cd0-49e3-4700-9341-95523a10fd49,2019-07-21 00:18:16.049660,2019-07-02 17:00:00,2cf94f65-6332-4b3b-9481-7e748671c54d,3992,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ho...,2019-07-21 00:13:11.547208,None,"Housing, Health, Energy, and Workers’ Rights C..."


`left_on` refers to the column name in the dataframe calling the operation.
In this case, the column to merge on is `body_id` in the events results.

Similarly, `right_on` refers to the column name in the dataframe to be passed to the operation.
In this case, the column to merge on is `id` in the bodies results.

`suffixes` is a tuple to use for adding suffixes to any columns with the same name between the two dataframes.
Commonly for CDP query results, these are columns such as `created`, which provide a `datetime` value for when that row was stored in the database.

Please refer to `pandas.DataFrame.merge` documentation for more details.

[reference](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge)

### Filtering

You may notice that the function: `select_rows_as_list` allows for additional parameters to be passed: `filters`, `order_by`, and `limit`. Unfortunately, at this time, `filters` is not available for the open access portions of the API. So while you can provide them to the function, they are not actually used. Because of this, you must do filtering on your end. Fortunately however, `pandas` works well for these types of operations.

[stackoverflow](https://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas)

In [8]:
fnc = "Finance and Neighborhoods Committee"
fnc_events = expanded_event_details.loc[expanded_event_details["name"] == fnc]
fnc_events

,agenda_file_uri,body_id,created_event,event_datetime,event_id,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri,created_body,description,name
21,http://legistar2.granicus.com/seattle/meetings...,0239d692-4514-4d62-bdcb-3b0c470dee5d,2019-07-20 23:59:10.237783,2019-06-26 14:00:00,a8fc8bc1-ac03-49ee-9e4b-67ba0c8a4fea,4010,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/fi...,2019-07-20 23:49:05.063092,None,Finance and Neighborhoods Committee
22,http://legistar2.granicus.com/seattle/meetings...,0239d692-4514-4d62-bdcb-3b0c470dee5d,2019-07-20 23:49:05.262303,2019-07-10 14:00:00,db75830f-caf9-4fbf-a8fa-0c018c048772,4025,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/fi...,2019-07-20 23:49:05.063092,None,Finance and Neighborhoods Committee
